prototyping of the interpolation data generation

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import random
import matplotlib.pyplot as plt
import random

import tensorflow as tf
import numpy as np
#import cmsisdsp as dsp
import random

#import sys
#sys.path.append('/home/mat/dev/cached_dilated_causal_convolutions/') 

# from cmsisdsp_py_version.block import Block
# from cmsisdsp_py_version.keras_model import create_dilated_model, create_strided_model
# from cmsisdsp_py_version.cached_block_model import CachedBlockModel, Regression
# from cmsisdsp_py_version.rolling_cache import RollingCache


In [ ]:
def parse(fname, w0n, w1n, w2n, invert_waves=[]):
    df_w = pd.read_csv(fname, sep=' ', names=['tri', w0n, w1n, w2n])
    df_w['n'] = range(len(df_w))
    for w in invert_waves:
        df_w[w] *= -1
    df_l = df_w.melt(id_vars='n', value_vars=['tri', w0n, w1n, w2n])
    return df_w, df_l
    
D = '/data2/cached_dilated_causal_convolutions/2d_embed_interp/wide_freq_range/24kHz'
tsrq_df_w, tsrq_df_l = parse(f"{D}/tri_sine_ramp_square.ssv", 'sine', 'ramp', 'square')
trqz_df_w, trqz_df_l = parse(f"{D}/tri_ramp_square_zigzag.ssv", 'ramp', 'square', 'zigzag', invert_waves=['ramp'])
tqzs_df_w, tqzs_df_l = parse(f"{D}/tri_square_zigzag_sine.ssv", 'square', 'zigzag', 'sine', invert_waves=['sine', 'square'])
tzsr_df_w, tzsr_df_l = parse(f"{D}/tri_zigzag_sine_ramp.ssv", 'zigzag', 'sine', 'ramp', invert_waves=['sine', 'zigzag'])

# tsrq_a = tsrq_df_w.to_numpy()
# tqzs_a = tqzs_df_w.to_numpy()


In [ ]:
len(tsrq_df_w)

In [ ]:
for df, fname in [(tsrq_df_l, '0_tsrq'), (trqz_df_l, '1_trqz'), 
                  (tqzs_df_l, '2_tqzs'), (tzsr_df_l, '3_tzsr')]:
    plt.figure(figsize=(16, 6))
    offset = 5500
    width = 500
    #df = tsrq_df_l
    window = (df['n']>offset) & (df['n']<offset+width)
    sns.lineplot(df[window], x='n', y='value', hue='variable')
    plt.savefig(fname)
    plt.clf()

In [ ]:
for o in range(600_000 // 50_000):    
    plt.figure(figsize=(16, 6))
    offset = o * 50_000
    width = 500
    df = tzsr_df_l
    window = (df['n']>offset) & (df['n']<offset+width)
    sns.lineplot(df[window], x='n', y='value', hue='variable')
    fname = f"CHECK_{o:05d}.png"
    plt.savefig(fname)
    plt.clf()

now we code picking a random point

In [ ]:
def wave_to_embed_pt(w):
    return {
        'sine': np.array([-1, -1]),
        'ramp': np.array([-1, 1]),
        'square': np.array([1, 1]),
        'zigzag': np.array([1, -1])
    }[w]
    
def moving_average(a, n=3):
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

def parse(fname, w0n, w1n, w2n):
    df_w = pd.read_csv(fname, sep=' ', names=['tri', w0n, w1n, w2n])
    df_w['n'] = range(len(df_w))
    #df_l = df_w.melt(id_vars='n', value_vars=['tri', w0n, w1n, w2n])
    return df_w.to_numpy() #, df_l
    
class WaveData(object):
    
    def __init__(self, wave0, wave1, data, pad_to_size, rescaling_factor):
        assert data.shape[1] == 3  # triangle + 2 waves
        self.wave0 = wave0
        self.wave1 = wave1
        self.embed_pt0 = wave_to_embed_pt(wave0)
        self.embed_pt1 = wave_to_embed_pt(wave1)
        self.data = data
        self.pad_to_size = pad_to_size
        self.rescaling_factor = rescaling_factor

    def sample(self, alpha, seq_len):
        
        # sample rows
        max_offset = len(self.data) - seq_len - 1
        random_offset = random.randint(0, max_offset)
        sample = self.data[random_offset:(random_offset+seq_len)]
        #print("sample", sample.shape)
        
        # interpolate sample
        interpolated_sample = (alpha * sample[:, 1]) + ((1-alpha) * sample[:, 2])
        
        # smooth with rolling average; can have some sharp boundaries
        # note: we need to pad to restore length ( do so with first element )
        N = 10
        interpolated_sample = moving_average(interpolated_sample, n=N)
        interpolated_sample = np.concatenate([[interpolated_sample[0]] * (N-1), interpolated_sample])

        # interpolate the embed points with the same alphas
        #print("self.embed_pt0", self.embed_pt0)
        #print("self.embed_pt1", self.embed_pt1)
        #print("alpha", alpha)
        interpolated_embed_pt = ( alpha * self.embed_pt0) + ((1-alpha) * self.embed_pt1)
        #print("interpolated_embed_pt", interpolated_embed_pt)
        interpolated_e0, interpolated_e1 = interpolated_embed_pt
        
        x = np.zeros((len(sample), self.pad_to_size), dtype=float)
        x[:, 0] = sample[:, 0] * self.rescaling_factor
        x[:, 1] = interpolated_e0 * self.rescaling_factor
        x[:, 2] = interpolated_e1 * self.rescaling_factor
        
        y = np.zeros((len(sample), self.pad_to_size), dtype=float)
        y[:, 0] = interpolated_sample * self.rescaling_factor
        
        return x, y        

    def as_tf_dataset(self, seq_len, max_samples):

        def gen():            
            num_samples_emitted = 0
            while True:
                yield self.sample(alpha=0.0, seq_len=seq_len)
                yield self.sample(alpha=1.0, seq_len=seq_len)
                yield self.sample(alpha=random.random(), seq_len=seq_len)
                num_samples_emitted += 3
                if num_samples_emitted > max_samples:
                    return
        
        return tf.data.Dataset.from_generator(
            gen, output_signature=(
                tf.TensorSpec(shape=(seq_len, self.pad_to_size), dtype=tf.float32),
                tf.TensorSpec(shape=(seq_len, self.pad_to_size), dtype=tf.float32)))


class Embed2DInterpolatedWaveFormData(object):
    
    def __init__(self,
                root_dir,
                rescaling_factor=1,
                pad_size=8):

        #D = 
        tsrq_a = parse(f"{root_dir}/tri_sine_ramp_square.ssv", 'sine', 'ramp', 'square')
        tqzs_a = parse(f"{root_dir}/tri_square_zigzag_sine.ssv", 'square', 'zigzag', 'sine')

        self.tsr = WaveData('sine', 'ramp',   tsrq_a[:,[0,1,2]], 
                            pad_to_size=pad_size, rescaling_factor=rescaling_factor)
        self.trq = WaveData('ramp', 'square', tsrq_a[:,[0,2,3]], 
                            pad_to_size=pad_size, rescaling_factor=rescaling_factor)                            
        self.tqz = WaveData('square', 'zigzag', tqzs_a[:,[0,1,2]], 
                            pad_to_size=pad_size, rescaling_factor=rescaling_factor)
        self.tzs = WaveData('zigzag', 'sine', tqzs_a[:,[0,2,3]], 
                            pad_to_size=pad_size, rescaling_factor=rescaling_factor)

        self.all_wave_data = [ self.tsr, self.trq, self.tqz, self.tzs ]

    def tf_dataset_for_split(self, split, seq_len, max_samples, waves=None):

        assert waves == None, "TODO"
        
        sampled_ds = tf.data.Dataset.sample_from_datasets(
            [wd.as_tf_dataset(seq_len, max_samples) for wd in self.all_wave_data]
        )

        if split == 'train':
            sampled_ds = sampled_ds.shuffle(1000)

        sampled_ds = sampled_ds.batch(1 if split=='test' else 64)

        return sampled_ds.prefetch(tf.data.AUTOTUNE)

        
def plot(x, y, fname):
    assert x.shape == y.shape    
    assert x.shape[1] == 8, x.shape
    df = pd.DataFrame()
    df['x'] = x[:,0]
    df['e0'] = x[:,1]
    df['e1'] = x[:,2]
    df['y'] = y[:,0]
    df['n'] = range(len(df))
    wide_df = pd.melt(df, id_vars=['n'], value_vars=['x', 'e0', 'e1', 'y'])
    plt.figure(figsize=(20, 10))
    p = sns.lineplot(wide_df, x='n', y='value', hue='variable')
    p.set_ylim((-2, 2))
    plt.savefig(fname)
    plt.clf()


data = Embed2DInterpolatedWaveFormData(
    root_dir='/data2/cached_dilated_causal_convolutions/2d_embed_interp/low_freq_range/24kHz')

for x, y in data.tf_dataset_for_split('train', seq_len=500, max_samples=100):    
    print(x.shape, y.shape)
    for b in range(len(x)):
        e0, e1 = x[b,0,1], x[b,0,2]
        fname = f"foo_{b:03d}.{e0}_{e1}.png"
        print(b, fname)
        plot(x[b], y[b], fname)
    break



In [ ]:
np.linspace(0, 1, 11)

In [ ]:
tzsr = tzsr_df_w.to_numpy()
tzsr[:10]

In [ ]:

wave_to_embed_pt = { 
    's': [-1, -1],
    'r': [-1, 1],
    'z': [1, 1],
    'q': [1, -1]
}

def moving_average(a, n=3):
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n
    
class WaveData(object):
    
    def __init__(self, waves, df, pad_to_size):
        self.waves = waves
        self.wave_to_column = { name: idx for idx, name in enumerate(waves) } 
        self.df = df
        self.pad_to_size = pad_to_size

    def _random_interpolation_from_sample(self, sample):
        
        interp_waves = self.waves[1:]
        random.shuffle(interp_waves)
#        print("interp_waves", interp_waves)

        # choose alpha for interps
        alpha1 = random.random()
        alpha2 = random.random()
#        print("alphas", alpha1, alpha2)

        # interpolate samples
        # TODO: constant power cross fade would be better here, otherwise we're getting
        #       an amplitude drop
        interp_columns = [self.wave_to_column[w] for w in interp_waves]
#        print("interp_columns", interp_columns)
        interpolated_samples = ( alpha1 * sample[:, interp_columns[0]]) + ((1-alpha1) * sample[:, interp_columns[1]])
        interpolated_samples = ( alpha2 * sample[:, interp_columns[2]]) + ((1-alpha2) * interpolated_samples)
        
        # smooth with rolling average; can have some sharp boundaries
        # note: we need to pad to restore length ( do so with first element )
        N = 10
        interpolated_samples = moving_average(interpolated_samples, n=N)        
        interpolated_samples = np.concatenate([[interpolated_samples[0]] * (N-1), interpolated_samples])
        
        # interpolate the embed points with the same alphas
        embed_pts = [np.array(wave_to_embed_pt[w]) for w in interp_waves]
#        print("interp_pts", interp_pts)
        interpolated_embed_pt = ( alpha1 * embed_pts[0]) + ((1-alpha1) * embed_pts[1])
#        print("interpolated_embed_pt", interpolated_embed_pt)
        interpolated_embed_pt = ( alpha2 * embed_pts[2]) + ((1-alpha2) * interpolated_embed_pt)
#        print("interpolated_embed_pt", interpolated_embed_pt)

        e0 = interpolated_embed_pt[0]  # e0
        e1 = interpolated_embed_pt[1]  # e1
        sy = interpolated_samples     
       
        return e0, e1, sy

    
    def _random_single_wave(self, sample):
        
        random_wave = random.choice(self.waves[1:])
        random_wave_col = self.wave_to_column[random_wave]
        embed_pt = wave_to_embed_pt[random_wave]
                
        e0 = embed_pt[0]   # e0
        e1 = embed_pt[1]   # e1
        sy = sample[:, random_wave_col]        
        
        return e0, e1, sy

    
    def sample(self, random_interpolation, seq_len):
        
        max_offset = len(self.df) - seq_len - 1
        random_offset = random.randint(0, max_offset)
        sample = self.df[random_offset:(random_offset+seq_len)]
        
        if random_interpolation:
            # interpolate randomly between the three waves
            e0, e1, sy = self._random_interpolation_from_sample(sample)            
        else:
            # just pick one of the waves to emit
            e0, e1, sy = self._random_single_wave(sample)

        x = np.zeros((len(sample), self.pad_to_size), dtype=float)
        x[:, 0] = sample[:, 0]
        x[:, 1] = e0
        x[:, 2] = e1
        y = np.zeros((len(sample), self.pad_to_size), dtype=float)  
        y[:, 0] = sy
        
        return x, y        

def parse(fname, w0n, w1n, w2n):
    df_w = pd.read_csv(fname, sep=' ', names=['tri', w0n, w1n, w2n])
    df_w['n'] = range(len(df_w))
    df_l = df_w.melt(id_vars='n', value_vars=['tri', w0n, w1n, w2n])
    return df_w, df_l

tsrq_df_w, tsrq_df_l = parse('../datalogger_firmware/tri_sine_ramp_square.ssv', 'sine', 'ramp', 'square')
trqz_df_w, trqz_df_l = parse('../datalogger_firmware/tri_ramp_square_zigzag.ssv', 'ramp', 'square', 'zigzag')
tqzs_df_w, tqzs_df_l = parse('../datalogger_firmware/tri_square_zigzag_sine.ssv', 'square', 'zigzag', 'sine')
tzsr_df_w, tzsr_df_l = parse('../datalogger_firmware/tri_zigzag_sine_ramp.ssv', 'zigzag', 'sine', 'ramp')

def make_dataset(wave_datas, seq_len):
    
    def gen():
        while True:
            for wd in wave_datas:
                yield wd.sample(random_interpolation=True, seq_len=seq_len)
                yield wd.sample(random_interpolation=False, seq_len=seq_len)
    
    
    return tf.data.Dataset.from_generator(
            gen, output_signature=(tf.TensorSpec(shape=(seq_len, 8), dtype=tf.float32),
                                   tf.TensorSpec(shape=(seq_len, 8), dtype=tf.float32)))    


wave_datas = [
    WaveData(['t', 's', 'r', 'q'], tsrq_df_w.to_numpy(), pad_to_size=8),
    WaveData(['t', 'r', 'q', 'z'], trqz_df_w.to_numpy(), pad_to_size=8),
    WaveData(['t', 'q', 'z', 's'], tqzs_df_w.to_numpy(), pad_to_size=8),
    WaveData(['t', 'z', 's', 'r'], tzsr_df_w.to_numpy(), pad_to_size=8)
]


In [ ]:
data = make_dataset(wave_datas, seq_len=1000).batch(20)

for x, y in data:
    print(x.shape, y.shape)
    break

In [ ]:
for i in range(len(x)):
    fig.clf()
    fig, (ax1, ax2) = plt.subplots(2)
    print(i, x[i][:5])
    ax1.plot(x[i])
    ax2.plot(y[i])
    ax1.set_ylim((-1, 1))
    ax2.set_ylim((-1, 1))
    plt.savefig(f"foo_{i:03d}.png")



we want to rebuild the dataset from tri, sine, ramp and tri, square, zigzag
to (embed0, embed1, tri) -> one of the other waves


In [ ]:
x = [1,2,3,4]

[x[0]]*5 + x

In [ ]:
data = tsr_df_w.to_numpy().astype(np.float32)

tri_to = {}
tri_to['sine'] = {}
tri_to['sine']['x'] = np.empty((len(data), 3), dtype=np.float32)
tri_to['sine']['x'][:,0] = 0  # x2
tri_to['sine']['x'][:,1] = 0  # x3
tri_to['sine']['x'][:,2] = data[:,0] # triangle
tri_to['sine']['y'] = np.expand_dims(data[:,1], -1) # sine

tri_to['ramp'] = {}
tri_to['ramp']['x'] = np.empty((len(data), 3), dtype=np.float32)
tri_to['ramp']['x'][:,0] = 0  # x2
tri_to['ramp']['x'][:,1] = 1  # x3
tri_to['ramp']['x'][:,2] = data[:,0] # triangle
tri_to['ramp']['y'] = np.expand_dims(data[:,2], -1) # ramp

data = tsz_df_w.to_numpy().astype(np.float32)

tri_to['square'] = {}
tri_to['square']['x'] = np.empty((len(data), 3), dtype=np.float32)
tri_to['square']['x'][:,0] = 1  # x2
tri_to['square']['x'][:,1] = 0  # x3
tri_to['square']['x'][:,2] = data[:,0] # triangle
tri_to['square']['y'] = np.expand_dims(data[:,1], -1) # square

tri_to['zigzag'] = {}
tri_to['zigzag']['x'] = np.empty((len(data), 3), dtype=np.float32)
tri_to['zigzag']['x'][:,0] = 1  # x2
tri_to['zigzag']['x'][:,1] = 1  # x3
tri_to['zigzag']['x'][:,2] = data[:,0] # triangle
tri_to['zigzag']['y'] = np.expand_dims(data[:,2], -1) # zigzag


In [ ]:
def split_train_val_test(d):
    assert 'x' in d
    assert 'y' in d
    assert len(d['x']) == len(d['y'])
    val_test_split_size = int(len(d['x']) * 0.1)  # 10% for val and test
    d['train'] = {}
    d['validate'] = {}
    d['test'] = {}
    for xy in ['x', 'y']:                
        d['train'][xy] = d[xy][:-2*val_test_split_size]        
        d['validate'][xy] = d[xy][-2*val_test_split_size:-val_test_split_size]        
        d['test'][xy] = d[xy][-val_test_split_size:]
        d.pop(xy)

for wave in ['sine', 'ramp', 'square', 'zigzag']:
    split_train_val_test(tri_to[wave])

In [ ]:
plt.plot(data[:,0][1000:2000])

In [ ]:
plt.plot(tri_to['ramp']['train']['x'][:,2][1000:2000])

In [ ]:
plt.plot(tri_to['ramp']['validate']['y'][1000:2000])

In [ ]:
plt.plot(tri_to['square']['test']['y'][1000:2000])

In [ ]:
plt.plot(tri_to['sine']['train']['y'][1000:2000])

In [ ]:
plt.plot(tri_to['zigzag']['validate']['y'][1000:2000])

In [ ]:
IN_D = 3    # 2d embedding, (0,1) and core triangle
OUT_D = 1   # output wave

# kernel size and implied dilation rate
K = 4                

# filters for Nth layer Kx1 and 1x1 convs
# [4, 3, 8, 8] @ 32kHz => 72%
# [4, 8, 8, 8] @ 32kHz => 82%
# [4, 8, 8, 12] @ 32kHz => 93%
# [8, 8, 8, 8] @ 32kHz => TOO MUCH
# [4, 4, 4] @ 96kHz => too much :/
# [2, 2, 4] @ 96kHz => too much :/
# [2, 2, 2] @ 96kHz => too much :/

FILTER_SIZES = [4, 4, 4, 4]

RECEPTIVE_FIELD_SIZE = K**len(FILTER_SIZES)

TEST_SEQ_LEN = RECEPTIVE_FIELD_SIZE
TRAIN_SEQ_LEN = RECEPTIVE_FIELD_SIZE * 5

print("RECEPTIVE_FIELD_SIZE", RECEPTIVE_FIELD_SIZE)
print("TRAIN_SEQ_LEN", TRAIN_SEQ_LEN)
print("TEST_SEQ_LEN", TEST_SEQ_LEN)

In [ ]:
from tensorflow.keras.optimizers import Adam

def masked_mse(y_true, y_pred):    
    assert len(y_true.shape) == 3, "expected (batch, sequence_length, output_dim)"
    assert y_true.shape == y_pred.shape
    
    # average over elements of y
    mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=-1)  
    
    # we want to ignore the first RECEPTIVE_FIELD_SIZE elements of the loss since they 
    # have been fed with left padded data
    mse = mse[:,RECEPTIVE_FIELD_SIZE:]     

    # return average over batch and sequence
    return tf.reduce_mean(mse)
    
def dataset_from(x, y, s):
    def gen():
        idxs = list(range(len(x)-TRAIN_SEQ_LEN-1))  # ~1.3M
        random.Random(1337).shuffle(idxs)
        if s == 'train':            
            idxs = idxs[:20_000]   # 200_000
        else:
            idxs = idxs[:500]   # 5_000
        for i in idxs:
            yield x[i:i+TRAIN_SEQ_LEN], y[i+1:i+1+TRAIN_SEQ_LEN]
                
    ds = tf.data.Dataset.from_generator(
        gen, output_signature=(tf.TensorSpec(shape=(TRAIN_SEQ_LEN, IN_D), dtype=tf.float32),
                               tf.TensorSpec(shape=(TRAIN_SEQ_LEN, OUT_D), dtype=tf.float32)))
    return ds

def datasets_for_split(s):
    return  [
        dataset_from(tri_to[wave][s]['x'], tri_to[wave][s]['y'], s) #.cache() #filename=f"tf_data_cache_{wave}")
        for wave in ['sine', 'ramp', 'square', 'zigzag']  
    ] 
    
train_ds = tf.data.Dataset.sample_from_datasets(datasets_for_split('train'))
train_ds = train_ds.batch(128).prefetch(tf.data.AUTOTUNE)

validate_ds = tf.data.Dataset.sample_from_datasets(datasets_for_split('validate'))
validate_ds = validate_ds.batch(128).prefetch(tf.data.AUTOTUNE)